# Beam finite element simulations with 4C

In this exercise, we will be creating, running and analyzing a beam finite element model, using the open source finite element software [4C](https://www.4c-multiphysics.org). 4C contains a variety of different finite element implementations for all kind of physical fields. In this exercise, we will exclusively be using reduced dimensional, geometrically exact beam elements, based on these two works:
- Jelenić, G., and Crisfield, M. A., 1999, “Geometrically Exact 3D Beam Theory: Implementation of a Strain-Invariant Finite Element for Statics and Dynamics,” Computer Methods in Applied Mechanics and Engineering, 171(1), pp. 141–171.
- Meier, C., 2016, “Geometrically Exact Finite Element Formulations for Slender Beams and Their Contact Interaction,” Dissertation, Technische Universität München.

For pre-processing, we will use the open source beam finite element generator [BeamMe](https://github.com/beamme-py/beamme). Post-processing is done directly within this notebook.

Note: The beam elements in 4C are developed for 3D analysis. However, in this exercise, we will only be considering plane examples.

In [ ]:
# We have to import the required packages, classes and functions.
# These imports will also be used in the exercises below, they only
# have to be included once.
import matplotlib.pyplot as plt
import numpy as np
from beamme.core.mesh import Mesh
from beamme.four_c.element_beam import Beam3rLine2Line2
from beamme.four_c.material import MaterialReissner

from utils.lecture_utils import (
    create_beam_mesh_line_2d,
    create_boundary_condition_2d,
    get_displacement_data,
    plot_beam_2d,
    run_four_c,
)

## Cantilever beam

In this first exercise, we will be creating the following cantilever beam model:

<img src="doc/cantilever.png" alt="Cantilever with force" width="400">

The beam has a length of $L = 0.3\text{m}$, a circular cross-section with radius $r = 0.01\text{m}$, a Young's modulus of $E = 2.1\cdot 10^{11}\text{N}/\text{m}^2$. The left end of the beam is clamped, while a load of $F = 10000\text{N}$ is applied at the right end in negative $y$-direction.

We use **two-noded linear** beam finite elements based on the geometrically exact **shear deformable** beam theory.


## Exercises

<div class="alert alert-info" role="alert">
  <strong>Exercise 2.1: Getting familiar with the simulation setup</strong>

  In this first exercise, take some time to familiarize yourself with the beam finite element simulation workflow.

  The code below sets up and runs a cantilever beam simulation using BeamMe for pre-processing and 4C as the solver.
  Run the simulation and inspect the deformed shape and internal force resultants.

  Then, modify the code and observe how the results change when you vary the following parameters:

  - Beam length $L$
  - Radius of the beam cross-section $r$
  - Young's modulus $E$
  - Load magnitude $F$
  - Number of finite elements along the beam `n_el`
  - Number of load steps `n_steps`

  For each modification:
  - Run the simulation
  - Inspect the deformed shape and cross-section force resultants
  - Briefly describe your observations

  <em>Note:</em> There are no “right” or “wrong” answers for this exercise. The goal is to build intuition for the influence of model parameters on the numerical solution.
</div>

In [ ]:
# Adjustable parameters
L = 0.3  # Length of the beam
radius = 0.01  # Radius of the beam cross-section
E = 2.1e11  # Young's modulus
F = 10000.0  # Applied force at the free end
n_el = 3  # Number of finite elements along the beam
n_steps = 1  # Number of load steps for applying the force

# In the beginning, we create the mesh container which will hold all elements,
# materials, and boundary conditions.
mesh = Mesh()

# Here we define the material properties for the beam elements.
# We use a Reissner beam material with specified radius, Young's modulus,
material = MaterialReissner(radius=radius, youngs_modulus=E)

# Next, we create a straight beam connecting two points in space.
# For spatial discretization, we use `n_el` linear two-noded beam elements.
beam_set = create_beam_mesh_line_2d(
    mesh, Beam3rLine2Line2, material, [0, 0], [L, 0], n_el=n_el
)

# To fix the cantilver beam, we apply Dirichlet boundary conditions to all
# positions and rotations at one node.
create_boundary_condition_2d(
    mesh, beam_set["start"], bc_type="dirichlet", directions=["x", "y", "theta"]
)

# At the other end of the beam, we apply a Neumann boundary condition
# representing a downward force in y-direction.
create_boundary_condition_2d(
    mesh, beam_set["end"], bc_type="neumann", directions=["y"], values=[-F]
)

# Now we can run the simulation. We have to give a unique name, in this case "ex_2_1",
# to store the results in a separate folder.
run_four_c(mesh=mesh, simulation_name="ex_2_1", n_steps=n_steps, tol=1e-10)

# Display the solution (referenced by the unique name "ex_2_1").
plot_beam_2d("ex_2_1")

<div class="alert alert-info" role="alert">
  <strong>Exercise 2.2: Cross-section force resultants</strong>

  On the right hand side of the plot, you can see the bending moment, shear force and axial force plotted over the beam length.
  For each element we plot the cross-section force resultants in the element center and not in the element nodes.

  According to the linear Bernoulli-Euler beam theory, the bending moment should be linear, with the maximum value at the fixed end and zero at the free end. The shear force should be constant and the axial force should be zero. 
  
  1. Why is the normal force not zero in the simulation results?
  1. Why is the shear force not constant in the simulation results?
  1. What assumptions of the Bernoulli–Euler beam theory are violated in the current simulation setup?
</div>

<div class="alert alert-success" role="alert">

  1. The normal force is not zero in the simulation results because the simulation is performed in a geometrically non-linear setting. This means that the equilibrium is fulfilled in the deformed state. The beam bends downwards, thus the vertical end load also has components in beam normal direction.
  1. The shear force is not constant for the same reason as described in the previous answer. The equilibrium is fulfilled in the deformed state, thus the vertical load is split up into normal and shear components. This split depends on the local orientation of the beam cross-section, which varies along the beam length, resulting in a non-constant shear force.
  1. The employed elements are based on a non-linear shear deformable beam theory. Compared to the linear Bernoulli-Euler beam theory, we have additional non-linear effects and the cross sections are not orthogonal to the beam centerline.
</div>


<div class="alert alert-info" role="alert">
  <strong>Exercise 2.3: Mesh convergence</strong>

  In this exercise, we investigate the influence of the spatial discretization on the numerical solution of a beam finite element model.

  1. Perform simulations using different numbers of beam elements, e.g.
     $$
     n_\mathrm{el} \in \{3, 5, 10, 20, 50, 100, 200\}.
     $$
     *Hint:* Use the provided function `get_tip_displacement(n_el)`. This function runs the simulation for a given number of elements and returns the displacement of the right end of the beam.
  1. Plot the norm of this displacement against the number of elements used in the discretization.
  1. Based on the results, discuss whether the solution appears to be converged and explain your observations.
</div>

In [ ]:
def get_tip_displacement(n_el):
    """Run the simulation for a given number of elements and return the
    displacement vector at the free end."""

    L = 0.3
    radius = 0.01
    E = 2.1e11
    F = 10000.0

    mesh = Mesh()
    material = MaterialReissner(radius=radius, youngs_modulus=E)

    beam_set = create_beam_mesh_line_2d(
        mesh, Beam3rLine2Line2, material, [0, 0], [L, 0], n_el=n_el
    )

    create_boundary_condition_2d(
        mesh, beam_set["start"], bc_type="dirichlet", directions=["x", "y", "theta"]
    )
    create_boundary_condition_2d(
        mesh, beam_set["end"], bc_type="neumann", directions=["y"], values=[-F]
    )

    run_four_c(mesh=mesh, simulation_name="ex_2_3", display_log=False)

    displacement = get_displacement_data("ex_2_3", point_coordinates=[L, 0])
    return displacement


number_of_elements = [3, 5, 10, 20, 50, 100, 200]
tip_displacement_norms = [
    np.linalg.norm(get_tip_displacement(n_el)) for n_el in number_of_elements
]

plt.plot(number_of_elements, tip_displacement_norms, "bx-")
plt.xlabel("$n_{el}$")
plt.ylabel("Displacement norm")
plt.title("Displacement vs. number of elements")
plt.grid(True)
plt.show()

<div class="alert alert-success" role="alert">

  3. The plot shows that the displacement converges to a fixed value as the number of elements increases. This indicates that the spatial discretization error decreases with increasing number of elements, leading to a more accurate solution.
</div>


<div class="alert alert-info" role="alert">
  <strong>Exercise 2.4: Bending of a beam to a half circle</strong>

  1. We now want to change the load at the free end to an external moment with magnitude $M = 2000\text{Nm}$. Adapt the code to achieve this (use 20 elements for the discretization).

     *Hint:* Look at the function `create_boundary_condition_2d` and its arguments. The `directions` argument specifies which components of the boundary condition are applied, while the `values` argument specifies the corresponding magnitudes.

  1. Increase the magnitude of the applied moment such that the beam exactly deforms into a half circle. 

     *Hint:* The moment of inertia for a circular cross-section is given as $I = \frac{\pi r^4}{4}$. The required moment to deform the beam into a half circle can then be calculated as $M = \pi \frac{EI}{L}$.

     *Hint:* This is a highly non-linear problem. You might need to increase the number of load steps `n_steps` to get a converged solution. 
</div>

In [ ]:
E = 2.1e11
radius = 0.01
L = 0.3
n_el = 20

moment_of_inertia = np.pi * radius**4 / 4
moment_magnitude = np.pi * E * moment_of_inertia / L

mesh = Mesh()
material = MaterialReissner(radius=radius, youngs_modulus=E)
beam_set = create_beam_mesh_line_2d(
    mesh, Beam3rLine2Line2, material, [0, 0], [L, 0], n_el=n_el
)
create_boundary_condition_2d(
    mesh, beam_set["start"], bc_type="dirichlet", directions=["x", "y", "theta"]
)
create_boundary_condition_2d(
    mesh,
    beam_set["end"],
    bc_type="neumann",
    directions=["theta"],
    values=[moment_magnitude],
)

run_four_c(mesh=mesh, simulation_name="ex_2_4", n_steps=10)

plot_beam_2d("ex_2_4")